Dataset: RAF-DB
The model architecture consists of three convolutional layers, each followed by max-pooling and dropout, ending with fully connected layers for classification.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix

In [4]:
# Define paths to your training and testing directories
train_dir = 'Datasets/RAF-DB/DATASET/train'
test_dir = 'Datasets/RAF-DB/DATASET/test'
# train_dir = 'Datasets/FER2013/train'
# test_dir = 'Datasets/FER2013/test'
# Set the image size and batch size
image_size = (224, 224)
#image_size = (48, 48)

batch_size = 32

# Create an ImageDataGenerator for data augmentation (optional)
train_datagen = ImageDataGenerator(
    rescale=1. / 255,  # Normalize pixel values
    horizontal_flip=True,  # Augmentation: horizontal flip
    # zoom_range=0.0,  # Augmentation: zoom (with current dataset not needed, since faces are centered)
    # rotation_range=10,  # Augmentation: rotation
    # width_shift_range=0.05,  # Augmentation: width shift (only 5% since faces are centered)
    # height_shift_range=0.05  # Augmentation: height shift (only 5% since faces are centered)
    brightness_range=[0.8, 1.2]
)

test_datagen = ImageDataGenerator(rescale=1. / 255)  # Only rescaling for test data

# Load images from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,  # Resize images to (48, 48) for uniformity
    batch_size=batch_size,  # Batch size for training
    #color_mode='grayscale',  # FER-2013 images are grayscale
    class_mode='categorical'  # For multi-class classification
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    #color_mode='grayscale',
    class_mode='categorical'
)

Found 12271 images belonging to 7 classes.
Found 3068 images belonging to 7 classes.


train_generator and test_generator are now ready to be used in model training and evaluation

In [5]:
# Define the model
model = Sequential()

# First Conv Block
model.add(Conv2D(224, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

# Second Conv Block
model.add(Conv2D(112, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

# Third Conv Block
model.add(Conv2D(56, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

# Flatten and Dense Layers
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))  # Assuming 7 emotions

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 224)     6272      
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 224)     0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 111, 111, 224)     0         
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 112)     225904    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 112)       0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 54, 54, 112)       0

In [ ]:
# Number of epochs to train for
epochs = 1

# Steps per epoch (usually the number of samples in the training set divided by the batch size)
steps_per_epoch = train_generator.samples // train_generator.batch_size

# Validation steps (usually the number of samples in the validation set divided by the batch size)
validation_steps = test_generator.samples // test_generator.batch_size

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=validation_steps
)

 87/383 [=====>........................] - ETA: 32:32 - loss: 1.7940 - accuracy: 0.3797

In [1]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, steps=np.ceil(test_generator.samples / test_generator.batch_size))
print("Test accuracy: ", test_accuracy)

# Predictions on the test set
test_generator.reset()
predictions = model.predict(test_generator, steps=np.ceil(test_generator.samples / test_generator.batch_size))
predicted_classes = np.argmax(predictions, axis=1)

# Since the generator omits some samples due to rounding down in 'steps', we trim 'true_classes' to match 'predicted_classes' length
true_classes = test_generator.classes
true_classes = true_classes[:len(predicted_classes)]

class_labels = list(test_generator.class_indices.keys())

# Classification report
print("Classification Report")
print(classification_report(true_classes, predicted_classes, target_names=class_labels, zero_division=0))

# Confusion Matrix
cm = confusion_matrix(true_classes, predicted_classes)
plt.figure(figsize=(10, 10))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.colorbar()
tick_marks = np.arange(len(class_labels))
plt.xticks(tick_marks, class_labels, rotation=45)
plt.yticks(tick_marks, class_labels)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()


NameError: name 'model' is not defined

Conducting error analysis
This can be done by examining misclassified examples, which can provide insights into what types of errors the model is making